https://colab.research.google.com/drive/1YORPWx4okIHXnjW7MSAidXN29mPVNT7F?usp=sharing#scrollTo=Aep1KMF6dqdm

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# T5-3b and T5-11B are supported!
# We need sharded weights otherwise we get CPU OOM errors
model_id=f"dim/persona_bot_2_28akcwik"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_8bit = AutoModelForSeq2SeqLM.from_pretrained(
    model_id, 
    device_map="auto", 
    load_in_8bit=True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto",
)

/home/dimweb/Desktop/deeppavlov/inference_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.device, model_8bit.device

(device(type='cuda', index=0), device(type='cuda', index=0))

In [12]:
model.get_memory_footprint() * 0.000001, model_8bit.get_memory_footprint() * 0.000001

(866.710884, 334.26500999999996)

In [3]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV2
from dimweb_persona_bot.hyperparameters.causal_modeling_hyperparameters import H2PersonaChatHyperparametersV1


hyperparameters = H2PersonaChatHyperparametersV1(
	chat_history_pair_length=7,
	persona_max_length=14,
	chat_max_length=25,
	debug_status=1,
)

bot2 = DialogBotV2(
	model=model,
	tokenizer=tokenizer,
	hyperparameters=hyperparameters,
	history=[],
	persona=[
		"i'm also a graduate student .",
		"i enjoy reading journals and guides related to psychology .",
		"my parents taught me survival skills .",
		"i walk dogs for a living .",
	],
)

bot2_int8 = DialogBotV2(
    model=model_8bit,
	tokenizer=tokenizer,
	hyperparameters=hyperparameters,
	history=[],
	persona=[
		"i'm also a graduate student .",
		"i enjoy reading journals and guides related to psychology .",
		"my parents taught me survival skills .",
		"i walk dogs for a living .",
	],
)

In [13]:
bot2.single_chat(message="What do you like?")

'i like dogs, i walk them for a living.'

In [14]:
bot2_int8.single_chat(message="What do you like?")

'i like dogs, i walk them for a living.'